In [ ]:
'''Baseball Data - A data set containing complete batting and pitching statistics from 1871 to 2014, plus fielding statistics,
standings, team stats, managerial records, post-season data, and more.

The following is an analysis of the batters and pitchers based on their performance. 

The data shown below helps to make critical decision during the transfer season and picking the required players to play for
their team. The data of the player performances have been taken and analysed below and the conclusions have been drawn at the 
end in graphical format.

'''

In [ ]:
'''
Here is a list of Abbreviations used in baseball:

yearID: Year
teamID: Team
Rank: Position in final standings
R: Runs scored
RA: Opponents runs scored
G: Games played
W: Wins
H: Hits by batters
BB: Walks by batters
HBP: Batters hit by pitch
AB: At bats
SF: Sacrifice flies
HR: Homeruns by batters
2B: Doubles
3B: Triples
'''

In [186]:
#importing the Files

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Loading the files from the directory
allstar=pd.read_csv("C:/Users/Prajval/baseball/AllstarFull.csv")
AwardsManagers=pd.read_csv("C:/Users/Prajval/baseball/AwardsManagers.csv")
AwardsPlayers=pd.read_csv("C:/Users/Prajval/baseball/AwardsPlayers.csv")
Batting=pd.read_csv("C:/Users/Prajval/baseball/Batting.csv")
Fielding=pd.read_csv("C:/Users/Prajval/baseball/Fielding.csv")
HallOfFame=pd.read_csv("C:/Users/Prajval/baseball/HallOfFame.csv")
Managers=pd.read_csv("C:/Users/Prajval/baseball/Managers.csv")
Pitching=pd.read_csv("C:/Users/Prajval/baseball/Pitching.csv")
Salaries=pd.read_csv("C:/Users/Prajval/baseball/Salaries.csv")
Teams=pd.read_csv("C:/Users/Prajval/baseball/Teams.csv")
Master=pd.read_csv("C:/Users/Prajval/baseball/Master.csv")


In [ ]:
'''

The Batting Average is defined by the number of hits divided by at bats. It can be calculated using the formula below:
BA = H/AB

On-base Percentage is a measure of how often a batter reaches base for any reason other than a fielding error, fielder's choice, dropped/uncaught third strike, fielder's obstruction, or catcher's interference. It can be calculated using the formula below:
OBP = (H+BB+HBP)/(AB+BB+HBP+SF)

Slugging Percentage is a measure of the power of a hitter. It can ve calculated using the formula below:
SLG = H+2B+(2*3B)+(3*HR)/AB


We will add these 3 measures to our teams DataFrame by running the following commands:

'''

In [188]:
#Batting Statistics

Batting['BA'] = Batting['H']/Batting['AB']
Batting['OBP'] = (Batting['H'] + Batting['BB'] + Batting['HBP']) / (Batting['AB'] + Batting['BB'] + Batting['HBP'] + Batting['SF'])
Batting['SLG'] = (Batting['H'] + Batting['2B'] + (2*Batting['3B']) + (3*Batting['HR'])) / Batting['AB']


In [189]:
'''
To predict the value of the batter, we have to consider the statistics based on attributes like Runs scored, Batting average, 
On base Percentage and so on.

We will use the linear regression model to verify which baseball stats are more important to predict runs.

We will build 3 different models: T
he first one will have as features OBP, SLG and BA. 
The second model will have as features OBP and SLG. 
The third one will have as feature BA only.

'''

import statsmodels.formula.api as sm

#First Model
runs_reg_model1 = sm.ols("R~OBP+SLG+BA",Batting)
runs_reg1 = runs_reg_model1.fit()
#Second Model
runs_reg_model2 = sm.ols("R~OBP+SLG",Batting)
runs_reg2 = runs_reg_model2.fit()
#Third Model
runs_reg_model3 = sm.ols("R~BA",Batting)
runs_reg3 = runs_reg_model3.fit()

'''Looking at the outputs of these models'''

runs_reg1.summary()
#runs_reg2.summary()
#runs_reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      R   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     5307.
Date:                Fri, 21 Jul 2017   Prob (F-statistic):               0.00
Time:                        07:46:55   Log-Likelihood:            -2.3476e+05
No. Observations:               50681   AIC:                         4.695e+05
Df Residuals:                   50677   BIC:                         4.696e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.6923      0.234     -2.959      0.003      -1.151      -0.234
OBP           71.3862      1.993     35.824      0.000      67.480      75.292
SLG           76.9252      1.196     64.344      0.000      74.582      79.268
BA           -92.9137      2.625    -35.392      0.000     -98.059     -87.768
==============================================================================
Omnibus:                     7264.033   Durbin-Watson:                   1.954
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            64781.149
Skew:                           0.410   Prob(JB):                         0.00
Kurtosis:                       8.478   Cond. No.                         30.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
'''
The first model has an Adjusted R-squared of 0.918, with 95% confidence interval of BA between -283 and 468. 

This is counterintuitive, since we expect the BA value to be positive. This is due to a multicollinearity between the variables.

The second model has an Adjusted R-squared of 0.919, and the last model an Adjusted R-squared of 0.500.

Based on this analysis, we could confirm that the second model using OBP and SLG is the best model for predicting Run Scored.
'''

In [190]:
Master["FullName"] = Master["nameFirst"].map(str) + Master["nameLast"]

In [191]:
'''Best Hitters Of all time'''

alltime = Batting
#alltime.set_index(['playerID'], inplace=True)
alltime = alltime.reset_index().groupby('playerID').sum()
alltime = alltime.sort_values(['OBP'], ascending=False)
alltime.drop(alltime[alltime.G < 100].index, inplace=True)

#Merge

alltime.reset_index(inplace=True)
mrg = pd.merge(alltime, Master, on='playerID', how='inner')

top = ['FullName', 'G', 'R', 'debut', 'AB', 'OBP', 'SLG', 'AB' ]
top = mrg[top]
top.head(20)

,FullName,G,R,debut,AB,OBP,SLG,AB
0,RickeyHenderson,3081,2295.0,1979-06-24,10961.0,11.476689,11.600811,10961.0
1,BarryBonds,2986,2227.0,1986-05-30,9847.0,9.713989,13.540737,9847.0
2,JimThome,2543,1583.0,1991-09-04,8422.0,9.502432,12.759343,8422.0
3,HaroldBaines,2830,1299.0,1980-04-10,9908.0,9.478566,12.001449,9908.0
4,PeteRose,3562,2165.0,1963-04-08,14053.0,9.340090,10.036042,14053.0
5,GarySheffield,2576,1636.0,1988-09-03,9217.0,9.194684,11.958558,9217.0
6,CarlYastrzemski,3308,1816.0,1961-04-11,11988.0,8.677707,10.537613,11988.0
7,JulioFranco,2527,1285.0,1982-04-23,8677.0,8.609726,9.635466,8677.0
8,RustyStaub,2951,1189.0,1963-04-09,9720.0,8.603506,10.131257,9720.0
9,TimRaines,2502,1571.0,1979-09-11,8872.0,8.601303,9.386902,8872.0


In [179]:
'''Best Hitters of Current Generation'''

new = Batting
new = new.drop(new[new.yearID < 2015].index)
new.drop(new[new.G < 50].index, inplace=True)
new = new.sort_values(['OBP'], ascending=False)

#Merge

new.reset_index(inplace=True)

mrge = pd.merge(new, Master, on='playerID', how='inner')

topnew = ['FullName', 'G', 'R', 'debut', 'AB', 'OBP', 'SLG', 'AB' ]
topnew = mrge[topnew]
topnew.head(26)


,FullName,G,R,debut,AB,OBP,SLG,AB
0,MikeDunn,72,0.0,2009-09-04,1.0,1.000000,1.000000,1.0
1,LiamHendriks,58,0.0,2011-09-06,0.0,1.000000,NaN,0.0
2,CarlosTorres,59,1.0,2009-07-22,1.0,1.000000,1.000000,1.0
3,RandyChoate,71,0.0,2000-07-01,0.0,1.000000,NaN,0.0
4,BryceHarper,153,118.0,2012-04-28,521.0,0.460245,0.648752,521.0
5,JoeyVotto,158,95.0,2007-09-04,545.0,0.458993,0.541284,545.0
6,MiguelCabrera,119,64.0,2003-06-20,429.0,0.440313,0.533800,429.0
7,PaulGoldschmidt,159,103.0,2011-08-01,567.0,0.435159,0.569665,567.0
8,MikeTrout,159,104.0,2011-07-08,575.0,0.401760,0.589565,575.0
9,AndrewMcCutchen,157,91.0,2009-06-04,566.0,0.401460,0.487633,566.0


In [70]:
'''Most valuable players of current generation'''

In [192]:
y = Batting
y = y.drop(y[y.yearID < 2010].index)
y.drop(y[y.G < 10].index, inplace=True)
y.drop(y[y.R < 10].index, inplace=True)
y = y.sort_values(['OBP'], ascending=False)
y

#Merge

y.reset_index(inplace=True)
y

mg = pd.merge(y, Salaries, on='playerID', how='left')
mg = mg.sort_values(['salary'], ascending=True)

mgnew = pd.merge(mg, Master, on='playerID', how='left')
mgnew = mgnew.set_index(['playerID'])
mgnew = mgnew.groupby(mgnew.index).first()
mgnew = mgnew.reset_index()

mn = ['FullName', 'G', 'R', 'debut', 'AB', 'OBP', 'SLG', 'AB' ,'salary']
mn = mgnew[mn]
mn.head(51)

,FullName,G,R,debut,AB,OBP,SLG,AB,salary
0,BobbyAbreu,142,54.0,1996-09-01,502.0,0.352740,0.364542,502.0,150000.0
1,JoseAbreu,154,88.0,2014-03-31,613.0,0.347305,0.502447,613.0,7000000.0
2,TonyAbreu,53,21.0,2007-05-22,138.0,0.301370,0.442029,138.0,393000.0
3,DustinAckley,85,22.0,2011-06-17,186.0,0.269608,0.365591,186.0,1700000.0
4,DavidAdams,43,10.0,2013-05-15,140.0,0.251656,0.285714,140.0,540000.0
5,MattAdams,108,46.0,2012-05-20,296.0,0.335423,0.503378,296.0,490000.0
6,JimAdduci,44,13.0,2013-09-01,101.0,0.238938,0.227723,101.0,501000.0
7,EhireAdrianza,53,10.0,2013-09-08,97.0,0.278846,0.298969,97.0,500500.0
8,NickAhmed,134,49.0,2014-06-29,421.0,0.275330,0.358670,421.0,508500.0
9,HanserAlberto,41,12.0,2015-05-29,99.0,0.237624,0.262626,99.0,NaN


In [195]:
'''Most Under-rated Players'''
#mn.set_index(['FullName'], inplace=True)
mn = mn.sort_values(['salary'], ascending=True)
mn

,FullName,G,R,debut,AB,OBP,SLG,AB,salary
799,OmarVizquel,108,36.0,1989-04-03,344.0,0.341146,0.331395,344.0,68000.0
762,JimThome,71,21.0,1991-09-04,206.0,0.351240,0.475728,206.0,100000.0
406,JasonKendall,118,39.0,1996-04-01,434.0,0.318182,0.297235,434.0,109000.0
834,GreggZaun,28,11.0,1995-06-24,102.0,0.350427,0.392157,102.0,109000.0
111,MikeCameron,45,18.0,1995-08-27,143.0,0.331288,0.419580,143.0,109000.0
127,LuisCastillo,86,28.0,1996-08-08,247.0,0.336806,0.267206,247.0,109000.0
393,ChipperJones,112,58.0,1993-09-11,387.0,0.377232,0.454780,387.0,109000.0
742,MikeSweeney,26,10.0,1995-09-04,52.0,0.310345,0.384615,52.0,109000.0
303,MarkGrudzielanek,30,10.0,1995-04-28,110.0,0.327731,0.272727,110.0,109000.0
652,ScottRolen,133,66.0,1996-08-01,471.0,0.357542,0.496815,471.0,109000.0


In [194]:
'''Best Pitchers of current generation(Based on ERA)'''


p = Pitching
#p.set_index(['playerID'], inplace=True)

#alltime = alltime.reset_index().groupby('playerID').sum()
#alltime = alltime.sort_values(['OBP'], ascending=False)
#p.drop(p[p.G < 50].index, inplace=True)
p = p.sort_values(['ERA'], ascending=False)

p = p.drop(p[p.yearID < 2010].index)

#Merge

p.reset_index(inplace=True)
pm = pd.merge(p, Master, on='playerID', how='inner')

top1 = ['FullName', 'yearID', 'G', 'R', 'debut', 'G', 'W', 'L', 'H', 'ER', 'ERA' ]
top1 = pm[top1]
top1.head(20)

,FullName,yearID,G,R,debut,G,W,L,H,ER,ERA
0,AlexHinshaw,2012,2,5,2008-05-15,2,0,0,4,5,135.00
1,AlexHinshaw,2012,31,14,2008-05-15,31,1,1,23,14,4.50
2,DavidCarpenter,2013,1,4,2012-04-13,1,0,0,2,4,108.00
3,DavidCarpenter,2015,4,3,2012-04-13,4,0,0,6,3,7.36
4,DavidCarpenter,2012,28,21,2012-04-13,28,1,2,42,21,4.76
5,DavidCarpenter,2014,1,0,2012-04-13,1,0,0,1,0,0.00
6,GarrettOlson,2012,1,4,2007-07-04,1,0,0,3,4,108.00
7,GarrettOlson,2010,35,20,2007-07-04,35,0,3,42,19,4.54
8,GarrettOlson,2011,4,1,2007-07-04,4,1,1,2,1,2.08
9,JorgeRondon,2015,2,11,2014-06-29,2,0,0,8,10,90.00


In [208]:
'''The Following graph shows the distribution of batters based on their salaries.
The Player names are distributed along the X-axis and the average salaries on the y-axis. 
The graph also indicates the runs scored by the batters in each season. As the graph moves from green to red in color, the 
runs scored by the player also increases. Thus the players having more red areas with their name have scored more runs.

Only players with OBP > 0.345 have been included. 
These players are the ones that are to be targeted when the transfer window opens up.


This Graph helps in making selections during the transfer season. The players can be chosen based on their 
market value and performances.
'''


'The Following graph shows the distribution of batters based on their salaries.\nThe Player names are distributed along the X-axis and the average salaries on the y-axis. \nThe graph also indicates the runs scored by the batters in each season. As the graph moves from green to red in color, the \nruns scored by the player also increases. Thus the players having more red areas with their name have scored more runs.\n\nOnly players with OBP > 0.345 have been included. \nThese players are the ones that are to be targeted when the transfer window opens up.\n\n\nThis Graph helps in making selections during the transfer season. The players can be chosen based on their \nmarket value and performances.\n'

In [207]:
%%HTML


<script type='text/javascript' src='https://us-east-1.online.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1523px; height: 678px;'><object class='tableauViz' width='1523' height='678' style='display:none;'><param name='host_url' value='https%3A%2F%2Fus-east-1.online.tableau.com%2F' /> <param name='site_root' value='&#47;t&#47;prajval' /><param name='name' value='Baseball-batting&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='showAppBanner' value='false' /><param name='showShareOptions' value='true' /></object></div>


In [209]:
'''The Following graph shows the distribution of pitchers based on their salaries.
The Player names are distributed along the X-axis and the average salaries on the y-axis. 
The graph also indicates the Strike Outs of pitchers. As the graph moves from green to red in color, the 
Strike Outs also increases. Thus the pitchers having a color more closer to red have more Strike outs.

The players are ranked based on their ERA. The graph shows the percentile of ERA of each player. 
The percentile of ERA of a Pitcher closer to 50  indicates that he is statistically stronger than the pitchers who are below 
him in the percentile score.

This Graph helps in making selections during the transfer season. The players can be chosen based on their 
market value and performances.

'''



'The Following graph shows the distribution of pitchers based on their salaries.\nThe Player names are distributed along the X-axis and the average salaries on the y-axis. \nThe graph also indicates the Strike Outs of pitchers. As the graph moves from green to red in color, the \nStrike Outs also increases. Thus the pitchers having a color more closer to red have more Strike outs.\n\nThe players are ranked based on their ERA. The graph shows the percentile of ERA of each player. \nThe percentile of ERA of a Pitcher closer to 50  indicates that he is statistically stronger than the pitchers who are below \nhim in the percentile score.\n\nThis Graph helps in making selections during the transfer season. The players can be chosen based on their \nmarket value and performances.\n\n'

In [211]:
%%HTML

<script type='text/javascript' src='https://us-east-1.online.tableau.com/javascripts/api/viz_v1.js'></script><div class='tableauPlaceholder' style='width: 1523px; height: 678px;'><object class='tableauViz' width='1523' height='678' style='display:none;'><param name='host_url' value='https%3A%2F%2Fus-east-1.online.tableau.com%2F' /> <param name='site_root' value='&#47;t&#47;prajval' /><param name='name' value='Baseball-pitcher&#47;Sheet5' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='showAppBanner' value='false' /><param name='showShareOptions' value='true' /></object></div>

In [ ]:
'''Hence the data has been analysed and conclusions are drawn. The data can be put to good use during the transfer season
where the managers will be hunting for upcoming talents who put great value to the team. '''